In [1]:
# code for loading the format for the notebook
import os

# path : store the current path to convert back to it later
path = os.getcwd()
os.chdir('../notebook_format')
from formats import load_style
load_style()

In [2]:
os.chdir(path)
import random
from collections import namedtuple

# Genetic Algorithm

**Genetic Algorithm (GA)** is a class of algorithms that is used for optimization. They find better answers to a "defined" question.

The intuition for GAs is that they generate a bunch of "answer candidates" and use some sort of feedback to figure out how close the candidate is to the "optimal" solution. During the process, far-from-optimal candidates gets dropped and are never seen again, while close-to-optimal candidates are combined with each other and maybe mutate slightly to see if they can get closer to optimal. The mutation is an attempt to modify the candidates from time to time to prevent the solution from getting stuck at the local optima.


## Chromosome

The "answer candidates" mentioned above are called **chromosomes**, which is the representation of a solution candidate.

**Chromosomes** mate and mutate to produce offspring. They either die due to survival of the fittest, or are allowed to produce offspring who may have more desirable traits and adhere to natural selection.

Suppose We're trying to optimize a very simple problem: trying to create a list of `N` numbers that equal `X` when summed together. If we set `N = 5` and `X = 200`, then our **chromosomes** will simply be a list with length of 5. Here are some possible **chromosomes** that could be solution candidates for our problem:

In [3]:
def chromosome( chromo_len, lower_bound, upper_bound ):
    return [ random.randint( lower_bound, upper_bound ) for _ in range(chromo_len) ]

chromo_len = 5
lower_bound = 0
upper_bound = 100
for i in range(3):
    print( chromosome( chromo_len, lower_bound, upper_bound ) )

[14, 51, 70, 62, 60]
[93, 72, 77, 59, 52]
[12, 8, 11, 82, 4]


## Population

The collection of all chromosomes is referred to as our **population**. When you run a GA, you don't just look at one chromosome at a time. You might have a **population** of 20 or 100 or 5,000 chromosomes going all at once.

In [4]:
def population( pop_size, chromo_len, lower_bound, upper_bound ):
    """
    creates a collection of chromosomes (i.e. a population)

    Parameters
    ----------
    pop_size : int
        number of chromosomes in the population

    chromo_len : int
        number of possible values per chromosome

    lower_bound, upper_bound : int
        lower_bound and upper_bound possible value of the randomly generated chromosome

    Returns
    -------
    (list) each element is a chromosome
    """
    return [ chromosome( chromo_len, lower_bound, upper_bound ) for _ in range(pop_size) ]

pop_size = 5
pop = population( pop_size, chromo_len, lower_bound, upper_bound )
pop

[[88, 91, 2, 53, 60],
 [35, 28, 8, 28, 81],
 [49, 93, 9, 85, 26],
 [60, 6, 64, 59, 53],
 [69, 39, 96, 62, 92]]

## Cost Function

But given all those randomly generated chromosomes, how do we measure the optimality of a chromosome and find the "correct" (or globally-optimum) chromosome?

The **cost function** (or error function, or fitness function as the inverse) is some sort of measure of the optimality of a chromosome. If we're calling it "fitness function" then we're shooting for higher scores, and if we're using "cost function" then we're looking for low scores.

In this case, we might define a **cost function** to be the absolute difference between the sum of the and the target number `X`. The reason we're using the square of the difference is so that we never end up with a negative cost, you can choose to square it if you want to.

In this case, since this problem is easy and contrived, we know that we're shooting for a cost of 0 (our sum of the numbers in the chromosome equals exactly to our target number) and that we can stop there. Sometimes that's not the case. Sometimes you're just looking for the lowest cost you can find, and need to figure out different ways to end the calculation. Other times you're looking for the highest fitness score you can find, and similarly need to figure out some other criteria to use to stop the calculation.

Using that rule as a cost function, we can calculate the costs of our population (each chromosomes).

In [5]:
def calculate_cost( chromosome, target ):
    """
    Determine the cost of an chromosome. lower is better

    Parameters
    ----------
    chromosome : list
        each chromosome of the entire population

    target : int
        the sum that the chromosomes are aiming for

    Returns
    -------
    (int) absolute difference between the sum of the numbers 
    in the chromosome and the target
    """
    summed = sum(chromosome)
    return abs(target - summed)

target = 200
for i in range(pop_size):
    graded = [ ( calculate_cost( p, target ), p ) for p in pop ]

for cost, chromo in graded:
    print( "chromo {}'s cost is {}".format( chromo, cost ) )

chromo [88, 91, 2, 53, 60]'s cost is 94
chromo [35, 28, 8, 28, 81]'s cost is 20
chromo [49, 93, 9, 85, 26]'s cost is 62
chromo [60, 6, 64, 59, 53]'s cost is 42
chromo [69, 39, 96, 62, 92]'s cost is 158


## Evolution - Crossover

Just like in evolution, you might be inclined to have the best and strongest chromosomes of the population mate with each other (The technical term for mating is **crossover**), with the hope that their offspring will be even healthier than either parent.

For each generation we'll retain a portion of the best performing chromosomes as judged by our cost/fitness function (the portion is a parameter that you can tune). These high-performers will be the parents of the next **generation**, or more intuitely, the next iteration.

Mating these parents is also very simple. You randomly pick two chromosomes, a male and a female (just a metaphor), and pick a point in the middle. This point can be dead-center if you want, or randomized if you prefer. Take that middle point (called a "pivot" point), and make two new chromosomes by combining the first half of one with the second half of the other and vice versa (It's usually recommended to use even numbers as your population size).

By repeating the mating step, we repopulate the population to its desired size for the next **generation**. e.g. if you take the top 30 chromosomes in a population of 100, then you'd need to create 100 new chromosomes by mating them.

## Evolution - Mutation

**Crossover** is the main step of how you get from one generation of chromosomes to the next, but it alone has a problem: If all you do is mate your candidates to go from generation to generation, you'll have a chance of getting stuck near a "local optimum, an answer that's pretty good but not necessarily the "global optimum" (the best you can hope for).

A GA would achieve very little if not for the combined effects of both **crossover** and **mutation**. **Crossover** helps discover more optimal solutions from already-good solutions, but it's the **mutation** that pushes the search for solutions in new directions.

**Mutation** is a completely random process by which you target an unsuspecting chromosome and blast it with just enough radiation to make one of its elements randomly change. How and when you mutate is up to you. e.g. If you choose a **mutation rate** of 0.1 (again any rate you want). Then if you randomly generated a number from 0 to 1 and if it happens to be below 0.1, the chromosome will **mutate**. 

As for the **mutation**, it can be randomly picking a element of the chromosome and add the number by 5, dividing the number by 2 or change it to a randomly generated number. Do whatever you want with it as long as it's relevant to the context of the problem. Like in the beginning of our problem, we set a maximum and minimum boundary when generating our initial values for each of the chromosome `min = 0 max = 100`. Then for our **mutation**, we can restrict our mutated number to be within this boundary (not really necessary here, but more problems that have indispensible boundaries this is crucial). And if we cross that border we can simply set it back to the value of that border (e.g. the mutated number is 102, we can simply squeeze it back to 100, our upper bound).

## Recap

The basic building blocks (parameters) of GA consists of: 

- **Chromosomes**. Representations of candidate solutions to your problem. They consist of the representation itself (in our case, a N element list)
- **Population**. A group of chromosomes. The population will remain the same size (you get to choose your population size.), but will typically evolve to better cost/fitness scores over time.
- **Cost/Fitness Function**. Used to evaluate your answer.
- The ability to **Crossover** and **Mutate**.

The population experiences **generations**. A typical GA takes the form of:

1. Initialize a population. Just fill it with completely random chromosomes that does not step over the boundary, if there is one.
2. Calculate the cost/fitness score for each chromosomes.
3. Sort the chromosomes by the user-defined cost/fitness score.
4. Retain a certain number of the parent chromosomes, where you get to pick the number of chromosome that will retain.
5. Mate the retained parent chromosomes to generate the children chromosomes. You can decide how you want to mate them. This process will keep going until the number of children chromosome is the same as the number of the original parent chromosome. 
6. Mutate the children chromosomes at random. Again, you can decide how to do this (resticted to the boundary).
7. Compare the parent chromosomes and the children chromosomes and choose the best ones (e.g. you have 100 parent chromosomes and generated 100 children chromosomes, you compare 200 chromosomes and retain the best 100 chromosomes). In other words, we're killing the poorly performed children.
8. If the algorithm has not met some kind of completion criteria, return to step 2 with the new chromosomes. The completion criteria for this example is pretty simple: stop when you get a cost of 0. But this isn't always the case. Sometimes you don't know the minimum achievable cost. Or, if you're using fitness instead of cost, you may not know the maximum possible fitness. In those cases you can stop the algorithm if the best score hasn't changed in 100 generations (iterations), or any other number depending on how much time are you willing to wait or the computation resources that you have, and use that as your answer.

---

Putting it all together, the code to evolve a generation can be implemented like this:

In [6]:
def evolve( pop, target, retain, mutate, chromo_len, lower_bound, upper_bound ):
    """
    evolution of the genetic algorithm

    Parameters
    ----------
    pop : list
        the initial population for this generation

    target : int
        the targeted solution

    retain : float
        the fraction of the best chromosome to retain. used to mate
        the children for the next generation

    mutate : float
        the probability that each chromosome will mutate
        
    chromo_len : int
        number of possible values per chromosome

    lower_bound, upper_bound : int
        lower_bound and upper_bound possible value of the randomly generated chromosome

    Returns
    -------
    children : list
        the crossovered and mutated population for the next generation

    generation_best : namedtuple( "cost", "chromo" )
        the current generation's best chromosome and its cost 
        (evaluated by the cost function)
    """

    # evolution :
    # take the proportion of the best performing chromosomes
    # judged by the calculate_cost function and these high-performers 
    # will be the parents of the next generation
    generation_info = namedtuple( "generation_info", [ "cost", "chromo" ] )
    graded = [ generation_info( calculate_cost( p, target ), p ) for p in pop ]
    graded = sorted(graded)
    parents = [ g.chromo for g in graded ]
    retain_len = int( len(parents) * retain )
    parents = parents[:retain_len]

    # the children_index set is used to
    # check for duplicate index1, index2. since
    # choosing chromosome ( a, b ) to crossover is the same
    # as choosing chromosome ( b, a ) 
    children = []
    children_index = set()
    desired_len = len(pop)
    parents_len = len(parents)

    # generate the the children (the parent for the next generation),
    # the children is mated by randomly choosing two parents and
    # mix the first half element of one parent with the later half 
    # element of the other
    while len(children) < desired_len:

        index1, index2 = random.sample( range(parents_len), k = 2 )

        if ( index1, index2 ) not in children_index:
            male   = parents[index1]
            female = parents[index2]
            pivot  = len(male) // 2
            child1 = male[:pivot] + female[pivot:]
            child2 = female[:pivot] + male[pivot:]
            children.append(child1)
            children.append(child2)
            children_index.add( ( index1, index2 ) )
            children_index.add( ( index2, index1 ) )
    
    # mutation :
    # randomly change one element of the chromosome
    for chromosome in children:
        if mutate > random.random():
            index_to_mutate = random.randrange(chromo_len)
            chromosome[index_to_mutate] = random.randint( lower_bound, upper_bound )
    
    # evaluate the children chromosome and retain the overall best
    graded_childen = [ generation_info( calculate_cost( p, target ), p ) for p in children ]
    graded.extend(graded_childen)
    graded = sorted(graded)
    generation_best = graded[0]
    children = [ g.chromo for g in graded[:desired_len] ]
    return children, generation_best

In [7]:
def ga( pop_size = 100, chromo_len = 5, lower_bound = 0, upper_bound = 100,
        retain = 0.5, mutate = 0.2, generation = 10, target = 200 ):

    # randomly generate the initial population
    pop = population( pop_size, chromo_len, lower_bound, upper_bound )

    # store the best chromosome and its cost for each generation,
    # so we can get an idea of when the algorithm converged
    generation_history = []
    for i in range(generation):
        pop, generation_best = evolve( pop, target, retain, mutate,
                                       chromo_len, lower_bound, upper_bound )
        generation_history.append(generation_best)
        print( "iteration {}'s best generation: {}".format( i + 1, generation_best ) )

    return generation_history

In [8]:
if __name__ == '__main__':
    random.seed(1234)
    ga1 = ga()

iteration 1's best generation: generation_info(cost=1, chromo=[21, 0, 85, 86, 7])
iteration 2's best generation: generation_info(cost=0, chromo=[85, 5, 29, 20, 61])
iteration 3's best generation: generation_info(cost=0, chromo=[85, 5, 29, 20, 61])
iteration 4's best generation: generation_info(cost=0, chromo=[24, 44, 45, 68, 19])
iteration 5's best generation: generation_info(cost=0, chromo=[24, 44, 45, 68, 19])
iteration 6's best generation: generation_info(cost=0, chromo=[24, 44, 45, 68, 19])
iteration 7's best generation: generation_info(cost=0, chromo=[24, 44, 45, 68, 19])
iteration 8's best generation: generation_info(cost=0, chromo=[17, 19, 64, 11, 89])
iteration 9's best generation: generation_info(cost=0, chromo=[17, 19, 64, 11, 89])
iteration 10's best generation: generation_info(cost=0, chromo=[17, 19, 64, 11, 89])


Even though we defined our generation number to be 10, we can reach our goal for only a small iteration count. Thus for problems where you don't the optimal answer, it's best to define a stopping criteria, instead of letting it run wild.

## Supplement

Recall that in the beginning, we said that Genetic Algorithm (GA) is a class of algorithms that is used for optimization. The term "is a class of algorithms" means that there're many different variations of GA. For example, in the evolution stage, instead of retaining a portion of the best performing chromosomes as judged by our cost/fitness function liked we mentioned above, we can throw darts to decide who gets to stay. 

By thowing darts, we are simply saying that we have some probability to select some of the lesser performing chromosomes in the current generation to be considered for the evolution stage. This *MIGHT* decrease our likelihood of getting stuck in the local optima. Example below:

In [9]:
"""
we have our chromosome and cost, the sum of the cost will be 
the probability's denominator, while the probability's numerator
is each chromo's cost (NOTE THAT THIS FOR MAXIMIZATION PROBLEM)
"""
denominator = 0
for cost, chromo in graded:
    denominator += cost

# the chromo, the cost and its probability of getting chosen as the
# parent used for evolution
for cost, chromo in graded:
    prob = cost / denominator
    print( "chromo {}'s cost is {} and it has a {} prob".format( chromo, cost, prob ) )

chromo [88, 91, 2, 53, 60]'s cost is 94 and it has a 0.25 prob
chromo [35, 28, 8, 28, 81]'s cost is 20 and it has a 0.05319148936170213 prob
chromo [49, 93, 9, 85, 26]'s cost is 62 and it has a 0.16489361702127658 prob
chromo [60, 6, 64, 59, 53]'s cost is 42 and it has a 0.11170212765957446 prob
chromo [69, 39, 96, 62, 92]'s cost is 158 and it has a 0.42021276595744683 prob


As you can see, we're not restricted to choose only the "best" chromosomes to be considered for the evolution stage. Each chromosome has a chance of being chosen, but we of course, are still in favor of the chromosomes that are performing well (has a higher probability of getting chosen).

In [10]:
"""
same idea for MINIMIZATION PROBLEM, except we have to 
use reciprocal costs, so that the lower the cost
the higher the probability
"""
denominator = 0
for cost, chromo in graded:
    denominator += 1 / cost

# the chromo, the cost and its probability of getting chosen as the
# parent used for evolution
for cost, chromo in graded:
    prob = ( 1 / cost ) / denominator
    print( "chromo {}'s cost is {} and it has a {} prob".format( chromo, cost, prob ) )

# you can confirm that the probability does add up to one if
# you're suspicious ^^

chromo [88, 91, 2, 53, 60]'s cost is 94 and it has a 0.09951079518662555 prob
chromo [35, 28, 8, 28, 81]'s cost is 20 and it has a 0.4677007373771401 prob
chromo [49, 93, 9, 85, 26]'s cost is 62 and it has a 0.15087120560552905 prob
chromo [60, 6, 64, 59, 53]'s cost is 42 and it has a 0.22271463684625717 prob
chromo [69, 39, 96, 62, 92]'s cost is 158 and it has a 0.05920262498444811 prob


## Reference

- [Genetic Algorithms: Cool Name & Damn Simple](http://lethain.com/genetic-algorithms-cool-name-damn-simple/)
- [Machine Learning: Introduction to Genetic Algorithms](https://www.burakkanber.com/blog/machine-learning-genetic-algorithms-part-1-javascript/)